In [ ]:
# Copyright 2025 Forusone

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Agent Development Kit


### Install adk

In [ ]:
%pip install --upgrade --quiet google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.7 MB/s eta 0:00:00


In [ ]:
# @title Project Information

PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"

MODEL_LIST = ["gemini-2.0-flash", \
              "gemini-2.5-pro-preview-03-25", \
              "gemini-2.5-flash-preview-04-17"]

MODEL_NAME = MODEL_LIST[1]
print(MODEL_NAME)

gemini-2.5-pro-preview-03-25


In [ ]:
# @title Authentication to access to GCP
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id = PROJECT_ID)

## Initial set up

## Agent on adk

In [ ]:
#@title Build a Agent with adk.
from google.adk.agents import Agent
from google.adk.tools import google_search

agent = Agent(
    name="search_assistant",
    model=MODEL_NAME,
    instruction="You are a helpful assistant. Answer user questions using Google Search when needed.",
    description="An assistant that can search the web.",
    tools=[google_search] # Use Google search tool
)


In [ ]:
from vertexai.preview.reasoning_engines import AdkApp

app = AdkApp(agent=agent)

for event in app.stream_query(
   user_id="shins",  # Required
   message="What is the exchange rate from US dollars to Korean currency today ?",
):
   print(event)
   print(event['content']['parts'][0]['text'])

{'content': {'parts': [{'text': 'Based on the search results for today, Saturday, April 26, 2025, here are the current exchange rates found for US Dollars (USD) to South Korean Won (KRW):\n\n*   **XE.com:** 1 USD = 1,437.42 KRW\n*   **Forbes Advisor (as of 10:05 UTC):** 1 USD = 1,438.36 KRW\n*   **Wise:** 1 USD = 1,438.69 KRW\n*   **Revolut:** Offers rates around 1 USD = 1,428.77 KRW to 1 USD = 1,439.41 KRW (rates may vary in-app and fees can apply). One source specifically stated 1 USD = 1,437.44 KRW.\n*   **The Economic Times:** 1 USD = 1,441.7 KRW\n\n**In summary:**\n\nAs of today, April 26, 2025, 1 US Dollar is equivalent to approximately **1,437 to 1,442 South Korean Won (KRW)**.\n\nPlease note that exchange rates fluctuate constantly and can vary slightly depending on the specific provider (like banks, currency exchange services, or money transfer platforms like Wise, Revolut, OFX, or Western Union). The exact rate you get will depend on when and where you perform the exchange, a

## End of notebook